In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_Maximizacao_Receita
C:\Users\flavi\Documents\GitHub\Projeto_Maximizacao_Receita


C:\Users\flavi\Documents\GitHub


In [2]:
# Importações necessárias 
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings

from src.config.paths import DADOS_AMOR_A_CAKES
from src.config.auxiliares_testes import analises_shapiro_levene

warnings.filterwarnings('ignore')

In [3]:
# Importando a dase de dados
df_vendas_EDA = pd.read_excel(DADOS_AMOR_A_CAKES)

# Define o número máximo de colunas a serem exibidas
pd.set_option('display.max_columns', 38)

df_vendas_EDA.head(2)

,data,loja,produto,categoria,custo_producao,preco_original,desconto_pct,preco_final,quantidade_vendida_dia,quantidade_vendida_mes,dia_semana,mes,feriado_flag,evento_especial,promocao_flag,promocao_tipo,promocao_inicio,promocao_fim,canal,regiao,bairro,perfil_loja,marca,tamanho_embalagem,tipo_produto,preco_competidor,indice_preco_competidor,gasto_marketing,campanha_id,estoque_disponivel,ruptura_flag,lead_time_reposicao,temperatura,chuva_mm,custo_logistico,custo_marketing_unitario,preco_minimo,margem_minima
0,2024-03-05,Tijuca,Torta de Limão,Doce,9.60,23.41,0.030,22.71,20,260,Tue,3,0,NaN,1,cashback,2024-01-01,2024-01-08,app,Zona Norte,Tijuca,bairro,SweetRJ,80g,ambiente,25.14,0.903341,69.31,CAMP-784,64,0,2,26.5,19.7,2.77,0.96,16.39,0.22
1,2024-05-22,Barra da Tijuca,Sanduíche Natural,Sanduíche,11.15,25.03,0.006,24.88,27,378,Wed,5,0,NaN,1,brinde,2024-01-01,2024-01-08,delivery,Zona Oeste,Barra da Tijuca,bairro,FreshSandwich,180g,ambiente,24.22,1.027250,77.43,CAMP-598,63,0,2,31.0,16.8,2.11,1.29,17.52,0.16


In [4]:
# Observar as informações do dataset
df_vendas_EDA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   data                      500 non-null    datetime64[ns]
 1   loja                      500 non-null    object        
 2   produto                   500 non-null    object        
 3   categoria                 500 non-null    object        
 4   custo_producao            500 non-null    float64       
 5   preco_original            500 non-null    float64       
 6   desconto_pct              500 non-null    float64       
 7   preco_final               500 non-null    float64       
 8   quantidade_vendida_dia    500 non-null    int64         
 9   quantidade_vendida_mes    500 non-null    int64         
 10  dia_semana                500 non-null    object        
 11  mes                       500 non-null    int64         
 12  feriado_flag          

In [5]:
# Obeservando a estatística descritiva da base de dados 
df_vendas_EDA.describe()

,data,custo_producao,preco_original,desconto_pct,preco_final,quantidade_vendida_dia,quantidade_vendida_mes,mes,feriado_flag,promocao_flag,promocao_inicio,promocao_fim,preco_competidor,indice_preco_competidor,gasto_marketing,estoque_disponivel,ruptura_flag,lead_time_reposicao,temperatura,chuva_mm,custo_logistico,custo_marketing_unitario,preco_minimo,margem_minima
count,500,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,491,491,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,2024-03-30 09:38:52.800000256,6.376160,14.379680,0.020744,14.080580,19.734000,338.016000,3.47400,0.026000,0.982000,2024-01-01 00:00:00.000000256,2024-01-08 00:00:00,14.150760,1.008220,102.922700,59.766000,0.008000,3.834000,28.185000,12.033600,1.770100,0.839000,10.065360,0.250640
min,2024-01-01 00:00:00,2.010000,4.910000,0.000000,4.790000,1.000000,13.000000,1.00000,0.000000,0.000000,2024-01-01 00:00:00,2024-01-08 00:00:00,4.180000,0.833333,10.420000,0.000000,0.000000,1.000000,25.000000,0.000000,0.500000,0.100000,3.440000,0.150000
25%,2024-02-14 18:00:00,4.057500,9.947500,0.012000,9.732500,11.000000,161.750000,2.00000,0.000000,1.000000,2024-01-01 00:00:00,2024-01-08 00:00:00,9.430000,0.897239,54.617500,30.000000,0.000000,2.000000,26.900000,5.700000,1.120000,0.487500,6.962500,0.200000
50%,2024-03-27 00:00:00,5.895000,12.655000,0.022000,12.445000,19.000000,319.500000,3.00000,0.000000,1.000000,2024-01-01 00:00:00,2024-01-08 00:00:00,12.660000,1.001364,107.530000,60.000000,0.000000,4.000000,28.100000,11.650000,1.810000,0.855000,8.860000,0.250000
75%,2024-05-16 06:00:00,8.640000,18.995000,0.030000,18.662500,29.000000,457.000000,5.00000,0.000000,1.000000,2024-01-01 00:00:00,2024-01-08 00:00:00,18.672500,1.094364,151.182500,90.000000,0.000000,5.000000,29.400000,18.800000,2.372500,1.180000,13.297500,0.300000
max,2024-06-29 00:00:00,11.970000,28.810000,0.040000,28.780000,40.000000,975.000000,6.00000,1.000000,1.000000,2024-01-01 00:00:00,2024-01-08 00:00:00,31.080000,1.246512,199.710000,120.000000,1.000000,7.000000,32.000000,24.900000,2.980000,1.500000,20.170000,0.350000
std,NaN,2.851449,5.975095,0.011466,5.845572,11.152219,213.317163,1.72838,0.159295,0.133084,NaN,NaN,6.096513,0.118715,55.345665,35.088255,0.089173,1.975411,1.681395,7.266178,0.719215,0.395625,4.182546,0.057594


In [6]:
# Observando valores ausentes na base 
df_vendas_EDA.isnull().sum()

data                          0
loja                          0
produto                       0
categoria                     0
custo_producao                0
preco_original                0
desconto_pct                  0
preco_final                   0
quantidade_vendida_dia        0
quantidade_vendida_mes        0
dia_semana                    0
mes                           0
feriado_flag                  0
evento_especial             487
promocao_flag                 0
promocao_tipo                 9
promocao_inicio               9
promocao_fim                  9
canal                         0
regiao                        0
bairro                        0
perfil_loja                   0
marca                         0
tamanho_embalagem             0
tipo_produto                  0
preco_competidor              0
indice_preco_competidor       0
gasto_marketing               0
campanha_id                   9
estoque_disponivel            0
ruptura_flag                  0
lead_tim

### Testes de Shapiro-Wilk e Levene

In [7]:
# Tratamento de valores ausentes em variáveis categóricas e seleção dinâmica
# Preenche nulos onde fizer sentido para análise descritiva e testes
df_vendas_EDA['evento_especial'] = df_vendas_EDA['evento_especial'].fillna('Nenhum')
df_vendas_EDA['promocao_tipo'] = np.where(
    df_vendas_EDA['promocao_flag'] == 0,
    df_vendas_EDA['promocao_tipo'].fillna('Sem_promocao'),
    df_vendas_EDA['promocao_tipo'].fillna('Desconhecido')
)
df_vendas_EDA['campanha_id'] = df_vendas_EDA['campanha_id'].fillna('Sem_campanha')

# Imputação de datas de promoção (tratando nulos)
df_vendas_EDA['promocao_inicio'] = pd.to_datetime(df_vendas_EDA['promocao_inicio'], errors='coerce')
df_vendas_EDA['promocao_fim'] = pd.to_datetime(df_vendas_EDA['promocao_fim'], errors='coerce')
mask_sem_promo = df_vendas_EDA['promocao_flag'] == 0
df_vendas_EDA.loc[mask_sem_promo, 'promocao_inicio'] = df_vendas_EDA.loc[mask_sem_promo, 'data']
df_vendas_EDA.loc[mask_sem_promo, 'promocao_fim'] = df_vendas_EDA.loc[mask_sem_promo, 'data']
mask_promo_missing = (df_vendas_EDA['promocao_flag'] == 1) & (df_vendas_EDA['promocao_inicio'].isna() | df_vendas_EDA['promocao_fim'].isna())
df_vendas_EDA.loc[mask_promo_missing & df_vendas_EDA['promocao_inicio'].isna(), 'promocao_inicio'] = df_vendas_EDA.loc[mask_promo_missing, 'data']
df_vendas_EDA.loc[mask_promo_missing & df_vendas_EDA['promocao_fim'].isna(), 'promocao_fim'] = df_vendas_EDA.loc[mask_promo_missing, 'promocao_inicio']

# Derivada útil: duração da promoção em dias
df_vendas_EDA['duracao_promocao_dias'] = (df_vendas_EDA['promocao_fim'] - df_vendas_EDA['promocao_inicio']).dt.days.clip(lower=0)

# Seleção programática de variáveis categóricas (objetos/categorias) + datas
variaveis_categoricas = df_vendas_EDA.select_dtypes(include=['object', 'category']).columns.tolist()
variaveis_categoricas += ['data', 'promocao_inicio', 'promocao_fim']

In [8]:
# Filtra somente variáveis numéricas para testes Shapiro-Wilk e Levene
df_numerico = df_vendas_EDA.drop(columns=variaveis_categoricas)

# Chamada da função com o DataFrame filtrado
analises_shapiro_levene(df_numerico)

# Salvando dataset limpo para uso no segundo notebook
from pathlib import Path
Path('dados').mkdir(exist_ok=True)
df_vendas_EDA.to_excel('dados/dataset_cafeterias_rj_limpo.xlsx', index=False)
print('Arquivo salvo em dados/dataset_cafeterias_rj_limpo.xlsx')

Teste de Shapiro-Wilk
estatistica_sw=0.940
custo_producao não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.947
preco_original não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.958
desconto_pct não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.948
preco_final não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.957
quantidade_vendida_dia não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.961
quantidade_vendida_mes não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.900
mes não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.144
feriado_flag não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.111
promocao_flag não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.954
preco_competidor não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.948
indice_preco_competidor não segue uma distribuição normal (valor p: 0.000)
estatistica_s

Arquivo salvo em dados/dataset_cafeterias_rj_limpo.xlsx


---
### Observação:

- Observamos pelo teste de **Shapiro-Wilk** & **Levene** que nenhuma vaiável do dataframe **df_vendas_AED** segue uma distribuição normal. <br>
- Isso é fundamental pois, com isso, sabemos que não podemos utilizar normalização no pré-processamento.

---